In [1]:
%cd /home/yoo/works/CGD

/home/yoo/works/CGD


In [3]:
# data/custom 폴더 하위에 있는 이미지 파일(jpg, JPG)를 하나의 텍스트 파일에 정리
# 파일 이름이 같은 경우에는 같은 라벨이 부여되도록 수정이 필요함(!!!)

import os

search_path = 'data/custom'
target_ext = '.jpg'

txt = open("{}/test.txt".format(search_path), 'w')
txt.write("image_id\tclass_id\tsuper_class_id\tpath")

idx = 0

for (path, dir, files) in os.walk(search_path):

    for filename in files:
        ext = os.path.splitext(filename)[-1]

        if path.split('/')[-1] != 'uncropped':
            
            if ext in ['.jpg', '.JPG']:
                idx += 1
                txt.write("\n")
                txt.write(f"_\t{idx}\t_\t{os.path.join(path, filename)}")
                # txt.write(f"_\t{idx}\t_\t{filename}")

txt.close()

In [5]:
# 데이터베이스 생성 (uncropped_data_dicts.pth)

import data_utils

data_path = 'data/custom'

data_utils.process_custom_data(data_path)

In [ ]:
# 데이터베이스 확인

import torch

pth_path = 'data/custom/uncropped_data_dicts.pth'

data_base = torch.load(pth_path)
data_base

In [22]:
import argparse

import pandas as pd
import torch
from thop import profile, clever_format
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import Model, set_bn_eval
from utils import recall, LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler

data_path = 'data'
data_name = 'custom'
crop_type = 'uncropped'
backbone_type = 'resnet50'
gd_config = 'SG'
feature_dim = 1536
smoothing = 0.1
temperature = 0.5
margin = 0.1
recalls = '1,2,4,8'
batch_size = 128

save_name_pre = '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(data_name, crop_type, backbone_type, gd_config, feature_dim,
                                                        smoothing, temperature, margin, batch_size)

test_data_set = ImageReader(data_path, data_name, 'test', crop_type)
test_data_loader = DataLoader(test_data_set, batch_size, shuffle=False, num_workers=8)
eval_dict = {'test': {'data_loader': test_data_loader}}

# model = Model(backbone_type, gd_config, feature_dim, num_classes=len(test_data_set.class_to_idx)).cuda()
model = Model(backbone_type, gd_config, feature_dim, num_classes=11318).cuda()

pretrained_model_path = 'results/sop_uncropped_resnet50_SG_1536_0.1_0.5_0.1_128_model.pth'
model.load_state_dict(torch.load(pretrained_model_path))

model.eval()
with torch.no_grad():
    # obtain feature vectors for all data
    for key in eval_dict.keys():
        eval_dict[key]['features'] = []
        for inputs, labels in tqdm(eval_dict[key]['data_loader'], desc='processing {} data'.format(key)):
            inputs, labels = inputs.cuda(), labels.cuda()
            features, classes = model(inputs)
            eval_dict[key]['features'].append(features)
        eval_dict[key]['features'] = torch.cat(eval_dict[key]['features'], dim=0)

data_base = {}
data_base['test_images'] = test_data_set.images
data_base['test_labels'] = test_data_set.labels
data_base['test_features'] = eval_dict['test']['features']
# torch.save(model.state_dict(), 'results/{}_model.pth'.format(save_name_pre))
torch.save(data_base, 'results/{}_data_base.pth'.format(save_name_pre))

processing test data: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


In [24]:
!python test.py \
--query_img_name 'data/custom/uncropped/24903648525.jpg' \
--data_base 'results/custom_uncropped_resnet50_SG_1536_0.1_0.5_0.1_128_data_base.pth' \
--retrieval_num 10

number 	 image_path 				 distance
1 	 data/custom/uncropped/25732645526.jpg 	 0.4202
2 	 data/custom/uncropped/24903715522.jpg 	 0.4648
3 	 data/custom/uncropped/32035855621.jpg 	 0.4799
4 	 data/custom/uncropped/25255997523.jpg 	 0.4868
5 	 data/custom/uncropped/34271791619.jpg 	 0.5056
6 	 data/custom/uncropped/32035866621.jpg 	 0.6053
7 	 data/custom/uncropped/22847254426.jpg 	 0.7456
8 	 data/custom/uncropped/21194929302.jpg 	 0.7632
9 	 data/custom/uncropped/21412657451.jpg 	 0.8244
10 	 data/custom/uncropped/36950336622.jpg 	 0.8389
